In [1]:
from torch import nn
import torch

device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

torch.device(device)

device(type='mps')

In [2]:
import pandas as pd

pdb_data = pd.read_csv("data/pdb/data_filtered_le_2048.csv")

In [3]:
pdb_data = pdb_data.dropna()

In [4]:
pad_token = "<pad>"
pad_token_val = 0

In [5]:
amino_acids = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V", "X"]

amino_acid_tokens = {aa : i + 1 for i, aa in enumerate(amino_acids)}
tokens_amino_acid = {i + 1 : aa for i, aa in enumerate(amino_acids)}

def amino_acid_tokenizer(amino_acid : str, amino_acid_tokens) -> torch.Tensor:
    return torch.tensor([amino_acid_tokens[aa] for aa in amino_acid], dtype=torch.long)

In [6]:
dssp_types = ["G", "H", "I", "T", "E", "B", "S", "P", "-"]

dssp_tokens = {dssp : i + 1 for i, dssp in enumerate(dssp_types)}
tokens_dssp = {i + 1 : dssp for i, dssp in enumerate(dssp_types)}

def dssp_tokenizer(dssp : str, dssp_tokens) -> torch.Tensor:
    return torch.tensor([dssp_tokens[dssp] for dssp in dssp], dtype=torch.long)

In [7]:
scrambled_data = pdb_data.sample(frac=1)
scrambled_data = scrambled_data.reset_index(drop=True)

data_size = len(scrambled_data)
train_size = int(data_size * 0.8)
validation_size = int(data_size * 0.1)
test_size = data_size - train_size - validation_size

train_data = scrambled_data[:train_size]
validation_data = scrambled_data[train_size:train_size + validation_size]
test_data = scrambled_data[train_size + validation_size:]

train_data = train_data.reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [8]:
train_input_df = train_data["sequence"]
train_output_df = train_data["secondary_structure"]

validation_input_df = validation_data["sequence"]
validation_output_df = validation_data["secondary_structure"]

test_input_df = test_data["sequence"]
test_output_df = test_data["secondary_structure"]

In [9]:
def create_dataset(input_df, output_df):
    tokenized_input = input_df.apply(amino_acid_tokenizer, amino_acid_tokens = amino_acid_tokens)

    input_tensor_seq = torch.nn.utils.rnn.pad_sequence(tokenized_input, batch_first = True, padding_value = pad_token_val)

    seq_length_tensor = torch.tensor([len(seq) for seq in input_df], dtype=torch.long)

    tokenized_output = output_df.apply(dssp_tokenizer, dssp_tokens = dssp_tokens)

    output_tensor_seq = torch.nn.utils.rnn.pad_sequence(tokenized_output, batch_first = True, padding_value = pad_token_val)    

    return torch.utils.data.TensorDataset(input_tensor_seq, output_tensor_seq, seq_length_tensor)

In [10]:
train_dataset = create_dataset(train_input_df, train_output_df)
validation_dataset = create_dataset(validation_input_df, validation_output_df)
test_dataset = create_dataset(test_input_df, test_output_df)

In [11]:
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [12]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Optimizer
from torch.utils.data import DataLoader

In [13]:
from typing import List, Tuple

In [14]:
def train(epoch : int,
          encoder : nn.Module,
          decoder : nn.Module,
          encoder_optimizer : Optimizer,
          decoder_optimizer : Optimizer,
          dataloader : DataLoader,
          decoder_criterion : nn.Module,
          teacher_forcing_ratio_decay : float = 0.,
          writer : SummaryWriter = None) -> float:
    encoder.train()
    decoder.train()

    running_loss = 0.

    for idx, (sequence_data, ss_data, seq_len) in enumerate(dataloader):
        sequence_data = sequence_data.to(device)
        ss_data = ss_data.to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_output, encoder_hidden = encoder(sequence_data, seq_len)
        decoder_output, _, _ = decoder(ss_data, encoder_output, encoder_hidden, teacher_forcing_ratio_decay)

        loss = decoder_criterion(decoder_output.view(-1, decoder_output.size(-1)), ss_data.view(-1))

        loss.backward()

        decoder_optimizer.step()
        encoder_optimizer.step()
        
        running_loss += loss.item()

        if writer is not None:
            writer.add_scalar("Loss/Train/Decoder/Batch", loss.item(), epoch * len(dataloader) + idx)
    
    avg_loss = running_loss / len(dataloader)

    if writer is not None:
        writer.add_scalar("Loss/Train/Decoder/Epoch", avg_loss, epoch)

    return avg_loss

In [15]:
def validate(epoch : int,
             encoder : nn.Module,
             decoder : nn.Module,
             dataloader : DataLoader,
             decoder_criterion : nn.Module,
             writer : SummaryWriter = None) -> float:
    encoder.eval()
    decoder.eval()

    running_loss = 0.

    with torch.no_grad():
        for idx, (sequence_data, ss_data, seq_len) in enumerate(dataloader):
            sequence_data = sequence_data.to(device)
            ss_data = ss_data.to(device)

            encoder_output, encoder_hidden = encoder(sequence_data, seq_len)
            decoder_output, _, _ = decoder(ss_data, encoder_output, encoder_hidden)

            decoder_loss = decoder_criterion(decoder_output.view(-1, decoder_output.size(-1)), ss_data.view(-1))

            running_loss += decoder_loss.item()

            if writer is not None:
                writer.add_scalar("Loss/Validation/Decoder/Batch", decoder_loss.item(), epoch * len(dataloader) + idx)
    
    avg_loss = running_loss / len(dataloader)

    if writer is not None:
        writer.add_scalar("Loss/Validation/Decoder/Epoch", avg_loss, epoch)

    return avg_loss

In [16]:
def test(encoder : nn.Module,
         decoder : nn.Module,
         dataloader : DataLoader,
         decoder_criterion : nn.Module) -> float:
    encoder.eval()
    decoder.eval()

    running_loss = 0.

    with torch.no_grad():
        for idx, (sequence_data, ss_data, seq_len) in enumerate(dataloader):
            sequence_data = sequence_data.to(device)
            ss_data = ss_data.to(device)

            encoder_output, encoder_hidden = encoder(sequence_data, seq_len)
            decoder_output, _, _ = decoder(ss_data, encoder_output, encoder_hidden)

            decoder_loss = decoder_criterion(decoder_output.view(-1, decoder_output.size(-1)), ss_data.view(-1))

            running_loss += decoder_loss.item()
                
    avg_loss = running_loss / len(dataloader)

    return avg_loss

In [17]:
from datetime import datetime

In [18]:
import pathlib

def train_model(encoder : nn.Module,
                decoder : nn.Module,
                device : torch.device,
                training_dataloader : DataLoader,
                validation_dataloader : DataLoader,
                test_dataloader : DataLoader,
                encoder_optimizer : Optimizer,
                decoder_optimizer : Optimizer,
                decoder_loss_fn : nn.Module,
                epochs : int,
                teacher_forcing_ratio_decay : float = 0.,
                model_name : str = "SecondCount") -> float:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    tensorboard_writer = SummaryWriter(log_dir=f"runs/{model_name}-{timestamp}")

    encoder.to(device)
    decoder.to(device)

    # Create models directory if it does not exist
    pathlib.Path(f"models/{model_name}-{timestamp}").mkdir(parents=True, exist_ok=True)

    best_total_loss = float("inf")

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        train_loss = train(
            epoch = epoch,
            encoder = encoder,
            decoder = decoder,
            encoder_optimizer = encoder_optimizer,
            decoder_optimizer = decoder_optimizer,
            dataloader = training_dataloader,
            decoder_criterion = decoder_loss_fn,
            teacher_forcing_ratio_decay = teacher_forcing_ratio_decay,
            writer = tensorboard_writer
        )
        validation_loss = validate(
            epoch = epoch,
            encoder = encoder,
            decoder = decoder,
            dataloader = validation_dataloader,
            decoder_criterion = decoder_loss_fn,
            writer = tensorboard_writer
        )

        print(f"Train Loss: {train_loss}, Validation Loss: {validation_loss}")

        if validation_loss < best_total_loss:
            best_total_loss = validation_loss

            torch.save(encoder.state_dict(), f"models/{model_name}-{timestamp}/encoder.pt")
            torch.save(decoder.state_dict(), f"models/{model_name}-{timestamp}/decoder.pt")

    test_loss = test(
        encoder = encoder,
        decoder = decoder,
        dataloader = test_dataloader,
        decoder_criterion = decoder_loss_fn
    )

    print(f"Test Loss: {test_loss}")

    tensorboard_writer.close()

    return test_loss


In [19]:
class Encoder(nn.Module):
    def __init__(self,
                 amino_acid_tokens : dict,
                 padding_token : int,
                 embedding_dim : int,
                 hidden_size : int,
                 bidirectional : bool = False,
                 num_layers : int = 1):
        super(Encoder, self).__init__()

        self.amino_acid_embedding = nn.Embedding(
            num_embeddings= len(amino_acid_tokens) + 1,
            embedding_dim = embedding_dim,
            padding_idx = padding_token
        )

        self.lstm = nn.LSTM(
            input_size = embedding_dim,
            hidden_size = hidden_size,
            batch_first = True,
            bidirectional = bidirectional,
            num_layers = num_layers
        )

    def forward(
            self, 
            input_seq : torch.Tensor,
            seq_len : torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        embedded_sequences = self.amino_acid_embedding(input_seq)

        packed_sequences = nn.utils.rnn.pack_padded_sequence(
            input = embedded_sequences, 
            lengths = seq_len.cpu(),
            batch_first = True,
            enforce_sorted = False
        )

        output, hidden = self.lstm(packed_sequences)

        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first = True)

        return output, hidden

In [20]:
class Attention(nn.Module):
    def __init__(self, hidden_size : int, num_layers : int = 1):
        super(Attention, self).__init__()

        self.Wa = nn.Linear(hidden_size * num_layers, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query : torch.Tensor, keys : torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        batch_size = query.size(0)
        query = query.reshape(batch_size, 1, -1)
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        attention_weights = torch.nn.functional.softmax(scores, dim=-1)
        context = torch.bmm(attention_weights, keys)

        return context, attention_weights

In [21]:
class Decoder(nn.Module):
    def __init__(self,
                 device : torch.device,
                 dssp_tokens : dict,
                 embedding_dim : int,
                 padding_token : int,
                 hidden_size : int,
                 encoder_bidirectional : bool = False,
                 num_layers : int = 1,
                 dropout : float = 0.1):
        super(Decoder, self).__init__()

        self.device = device
        self.padding_token = padding_token
        self.dssp_tokens = dssp_tokens
        self.encoder_bidirectional = encoder_bidirectional
        
        if encoder_bidirectional:
            hidden_size *= 2

        self.attention = Attention(hidden_size = hidden_size, num_layers = num_layers)
        self.dssp_embedding = nn.Embedding(num_embeddings = len(dssp_tokens) + 1, 
                                           embedding_dim = embedding_dim,
                                           padding_idx = padding_token)
        self.lstm = nn.LSTM(input_size = embedding_dim + hidden_size,
                            hidden_size = hidden_size,
                            batch_first = True,
                            num_layers = num_layers)
        self.fc = nn.Linear(in_features = hidden_size,
                            out_features = len(dssp_tokens) + 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,
                ss_tensor : torch.Tensor,
                encoder_outputs : torch.Tensor,
                encoder_hidden : torch.Tensor,
                teacher_forcing_ratio : float = 0.) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        batch_size, sequence_length = ss_tensor.size()

        decoder_input = torch.empty(batch_size, 1, dtype=torch.long).fill_(self.padding_token).to(self.device)
        if self.encoder_bidirectional:
            encoder_even_hidden = (encoder_hidden[0][::2], encoder_hidden[1][::2])
            encoder_odd_hidden = (encoder_hidden[0][1::2], encoder_hidden[1][1::2])
            encoder_hidden = (torch.cat((encoder_even_hidden[0], encoder_odd_hidden[0]), dim=-1), torch.cat((encoder_even_hidden[1], encoder_odd_hidden[1]), dim=-1))
        
        decoder_hidden = encoder_hidden

        decoder_outputs = []
        attentions = []

        for idx in range(sequence_length):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, encoder_outputs, decoder_hidden)
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if teacher_forcing_ratio > 0 and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input = ss_tensor[:, idx].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        ans = torch.stack(decoder_outputs, dim=1)
        ans = torch.nn.functional.log_softmax(ans, dim=-1)
        attentions = torch.cat(attentions, dim=1)
        return ans, decoder_hidden, attentions
    
    def forward_step(self, 
                     input_tensor : torch.Tensor, 
                     encoder_outputs : torch.Tensor, 
                     encoder_hidden : torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        embedded_tensor = self.dropout(self.dssp_embedding(input_tensor))

        query = encoder_hidden[0].permute(1, 0, 2)
        context, attention_weights = self.attention(query, encoder_outputs)
        input = torch.cat((embedded_tensor, context), dim=2)

        output, encoder_hidden = self.lstm(input, encoder_hidden)
        output = self.fc(output)
        return output, encoder_hidden, attention_weights

In [22]:
#encoder = Encoder(dssp_tokens, 32, 64, device, True, 4)
encoder = Encoder(
    amino_acid_tokens = amino_acids,
    padding_token = pad_token_val,
    embedding_dim = 32,
    hidden_size = 16
)
# decoder = Decoder(dssp_tokens, 32, 64, device, 0.1, True, 4)
decoder = Decoder(
    device = device,
    dssp_tokens = dssp_types,
    embedding_dim = 32,
    padding_token = pad_token_val,
    hidden_size = 16
)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())


In [23]:
decoder_loss = nn.NLLLoss()

In [24]:
teacher_forcing_ratio_decay = 0.5

In [25]:
train_model(
    encoder = encoder,
    decoder = decoder,
    device = device,
    training_dataloader = training_dataloader,
    validation_dataloader = validation_dataloader,
    test_dataloader = test_dataloader,
    encoder_optimizer = encoder_optimizer,
    decoder_optimizer = decoder_optimizer,
    decoder_loss_fn = decoder_loss,
    epochs = 10
)

Epoch 1/10


KeyboardInterrupt: 